In [1]:
# importing packages

import sys
print sys.version

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

# special matplotlib argument for improved plots
from matplotlib import rcParams


2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) 
[GCC 4.2.1 (Apple Inc. build 5577)]


In [6]:
df=pd.read_csv("1990-2015.csv")

In [7]:
df.head()

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,conm,...,chq,cshiq,cstkq,dlcq,dlttq,ppentq,pstkq,wcapq,costat,sic
0,1003,01/31/1990,1989,4,INDL,C,D,STD,ANTQ,A.A. IMPORTING CO INC,...,NaN,NaN,0.268,4.449,0.076,0.654,0,-1.259,I,5712
1,1003,04/30/1990,1990,1,INDL,C,D,STD,ANTQ,A.A. IMPORTING CO INC,...,NaN,NaN,0.268,4.450,0.068,0.591,0,-1.970,I,5712
2,1003,07/31/1990,1990,2,INDL,C,D,STD,ANTQ,A.A. IMPORTING CO INC,...,NaN,NaN,0.268,4.451,0.059,0.388,0,-3.059,I,5712
3,1003,10/31/1990,1990,3,INDL,C,D,STD,ANTQ,A.A. IMPORTING CO INC,...,NaN,NaN,0.268,4.452,0.051,0.346,0,-3.437,I,5712
4,1004,02/28/1990,1989,3,INDL,C,D,STD,AIR,AAR CORP,...,NaN,NaN,16.070,32.551,72.606,53.428,0,188.100,A,5080


In [ ]:
# basic function to calculate ratio 1 across a row
# ratio 1 = EBIT / (NFA + NWC)
# EBIT = earnings before interest and taxes, after subtracting depreciation and amortization
# NFA = net fixed assets = net book value of Property Plant and Equipment (PPENT)
# NMW = net working capital = working capital - cash = WCAPQ - CHQ

def ratio_one(row):
    ratio=row['EIBT']

In [ ]:
# basic function to calculate ratio 2 across a row